# Parsing du document texte

In [1]:
import codecs
import pandas as pd


# Ouvrir le fichier en lecture seule
file = codecs.open('Planning_JO2024_v3.txt', "r", "utf-8")
# utiliser readlines pour lire toutes les lignes du fichier
# La variable "lignes" est une liste contenant toutes les lignes du fichier
lines = file.readlines()
# fermez le fichier après avoir lu les lignes
file.close()


words = []
for line in lines:
    words += line.split()
    
demarq1_discipline = "calendrier reste susceptible de modification jusqu'à la conclusion des Jeux Olympiques.".split()
demarq2_discipline = "A noter que l'ordre des matchs sera déterminé pendant les Jeux en fonction du temps de repos des athlètes, des demandes des diffuseurs et d'autres paramètres qu'il n'est pas possible d'anticip".split()
discipline = 'ATHLETISME'

data = []
i = 0
while i < (len(words)):
    # print("Passage de boucle")
    word = words[i]        
    if words[i].upper() == "JOUR":
        if len(words) > i + 4:
            jour = words[i+1]
            date = words[i+2] + " " + words[i+3] + " " + words[i+4]
            i +=4
            # print("jour ", jour)
        else:
            print("erreur")

    # discipline
    if words[i:i+len(demarq1_discipline)] == demarq1_discipline:
        i+=len(demarq1_discipline)
        if words[i:i+len(demarq2_discipline)] == demarq2_discipline:
            i+=len(demarq2_discipline)
        discipline = ""
        espace = ""
        while words[i] != "Calendrier":
            discipline += espace + words[i]
            espace = " "
            i += 1
            # print("discipline ", discipline)
    
    if words[i] == "Session":
        session = words[i+1]
        i += 2
        # print("session ", session)
    if words[i] == "Début":
        debut = words[i+2]
        i += 3
    if words[i] == "Fin":
        fin = words[i+2]
        i += 3
        lieu_epreuve = ""
        espace = ""
        while words[i] != "Nom":
            lieu_epreuve += espace + words[i]
            espace = " "
            i += 1
            # print('lieu ', lieu_epreuve)
    nom_epreuve = ""
    if words[i:i+4] == ['Nom', 'de', "l'épreuve/des", 'épreuves']:
        i += 4
        espace = ""
        while len(words) > i and words[i].upper() not in ["JOUR", "SESSION"] and words[i:i+2] != ['Le', 'calendrier']:
            nom_epreuve += espace + words[i]
            espace = " "
            # print(words[i])
            i += 1
        i-=1
        # print(nom_epreuve)
        data.append([jour, date, discipline, session, debut, fin, lieu_epreuve, nom_epreuve])
    i += 1

def traitement_discipline(string):
    if len(string) == 0:
        return ""
    else:
        return string[0].upper() + string[1:].lower()


df = pd.DataFrame(data, columns=['num_jour', 'date', 'discipline', 'num_session', 'debut_epreuve', 'fin_epreuve', 'lieu_epreuve', 'nom_epreuve'])
df['discipline'] = df['discipline'].apply(traitement_discipline)
#df.to_csv('../data/session_planning_olympics.csv')

In [2]:
df

,num_jour,date,discipline,num_session,debut_epreuve,fin_epreuve,lieu_epreuve,nom_epreuve
0,6,Jeudi 1 Août,Athletisme,ATH01,07:30,11:05,Trocadéro,"20 km marche - hommes, finale 20 km marche - f..."
1,7,Vendredi 2 Août,Athletisme,ATH02,10:00,13:00,Stade de France,"Décathlon - 100 m, hommes Lancer de marteau - ..."
2,7,Vendredi 2 Août,Athletisme,ATH03,17:40,22:00,Stade de France,"Décathlon - saut en hauteur, hommes 5 000 m - ..."
3,8,Samedi 3 Août,Athletisme,ATH04,10:00,16:00,Stade de France,"Décathlon - 110 m haies, hommes Saut à la perc..."
4,8,Samedi 3 Août,Athletisme,ATH05,19:00,22:00,Stade de France,"1 500 m - hommes, Repêchages Lancer de poids -..."
...,...,...,...,...,...,...,...,...
752,14,Vendredi 9 Août,Water polo,WPO32,18:00,21:00,Paris La Défense Arena,"Hommes, match de classement Hommes, demi-finale"
753,15,Samedi 10 Août,Water polo,WPO33,09:00,12:00,Paris La Défense Arena,"Femmes, match de classement Femmes, match pour..."
754,15,Samedi 10 Août,Water polo,WPO34,14:00,21:00,Paris La Défense Arena,"Femmes, match de classement Femmes, match pour..."
755,16,Dimanche 11 Août,Water polo,WPO35,09:00,12:00,Paris La Défense Arena,"Hommes, match de classement Hommes, match pour..."


# Parsing noms d'epreuves

In [4]:
import json

# separateurs = [' - ', ', '], 
# noms = ['epreuve', 'genre', 'etape']


def parser(texte, separateurs, noms):
    #print(texte)
    epreuves = {'content' : []}
    # epreuves = []
    # epreuve, genre, etape = '', '', ''
    dico_epreuve = {}
    i, i0 = 0, 0
    while i < len(texte):
        #print(i)
        for n in range(len(separateurs)):
            separateur = separateurs[n]
            #print("Chgt separateur : ", separateur)
            while i < len(texte):
                if i + len(separateur) < len(texte) and texte[i:i+len(separateur)] == separateur:
                    dico_epreuve[noms[n]] = texte[i0:i]
                    #print(texte[i0:i])
                    i+=len(separateur)
                    i0 = i
                    break
                i+=1
        #print(i0)
        while i <= len(texte):
            if  i == len(texte) or nom_epreuve(texte, i):
                dico_epreuve[noms[-1]] = texte[i0:i]
                i0 = i
                break
            i+=1
        epreuves['content'].append(dico_epreuve.copy())#json.dumps(dico_epreuve.copy()))
        # epreuves.append(dico_epreuve.copy())
        i+=1
    return json.dumps(epreuves)

noms_sports = [
    "20 km",
    "100 m",
    "200 m",
    "1 500 m",
    "5 000 m",
    "800 m",
    "10 000 m",
    "3 000 m",
    "400 m",
    "110 m",
    "50kg",
    "51kg",
    "54kg",
    "60kg",
    "66kg",
    "75kg",
    "63.5kg",
    "63,5kg",
    "80kg",
    "92kg",
    "71kg",
    "57kg",
    "+92kg"
]

pas_noms_epreuve = [
    "Femmes",
    "Hommes",
    "Repêchage",
    "Qualifications",
    "Eliminatoires",
    "Quart de finale",
    "Demi-finale",
    "Match pour la médaille",
    "Série",
    "Course pour"
]

# inclure pas nom d'épreuve
def nom_epreuve(texte, i):
    if texte[i].isupper():
        for nom in pas_noms_epreuve:
            if i + len(nom) <= len(texte) and texte[i:i+len(nom)] == nom:
                return False            
        if i + 3 < len(texte) and texte[i+1] != " " and texte[i+2] != " " and texte[i+3] != " " :
            return True
    for nom in noms_sports:
        if i + len(nom) <= len(texte) and texte[i:i+len(nom)] == nom:
            return True
    return False

noms_etapes = ["eliminatoires", "repêchage"] # --> enlever majuscule

# Rajouter condition sur nom sport: plus de 3 lettres
# pour tej les "finale A/B" et autres

cles_parsers = {
    'ATH ROW BMF CTR CLB FEN ARC SHO BOX CSL CSP WRE JUD SWM' : {
            'separateurs' : [' - ', ', '], 
            'noms' : ['epreuve', 'genre', 'etape']
    },
    'BKB BMX BK3 RU7 BKG GTR HBL HOC SRF TTE VVO WPO' : {
        'separateurs' : [', '], 
        'noms' : ['genre', 'etape']
    },
    'CRD OWS DIV MTB WLF' : {
        'separateurs' : [' - '], 
        'noms' : ['epreuve', 'genre']
    },
    'GLF MPN SAL' : {
        'separateurs' : [' - '], 
        'noms' : ['epreuve', 'etape']
    },
    'FBL' : {
        'separateurs' : [', ', ' - '], 
        'noms' : ['genre', 'etape', 'epreuve']
    },
    'GAR VBV' : {
        'separateurs' : [' - '], 
        'noms' : ['etape', 'genre']
    },
    'GRY BDM' : {
        'separateurs' : [', '], 
        'noms' : ['epreuve', 'etape']
    },
    'SWA EQU TKW TEN SKB' : {
        'separateurs' : [], 
        'noms' : ['epreuve']
    },
    'WLF TRI' : {
        'separateurs' : [], 
        'noms' : ['genre']
    },
}


def match_cle_parser(code_disc, cles_parsers):
    for cle in cles_parsers.keys():
        if code_disc in cle:
            return cles_parsers[cle]
    print(f"Cette discipline n'est pas prise en charge : {code_disc}")

def parsing_epreuve(row):
    texte = row.nom_epreuve
    if len(str(row.num_session)) > 2:
        disc = str(row.num_session)[:3]
        cle_parser = match_cle_parser(disc, cles_parsers)
        if cle_parser is not None:
            return parser(texte, cle_parser['separateurs'], cle_parser['noms'])

# disc_analysees = ['ATHLETISME', 'AVIRON', 'BASKETBALL', 'BASKETBALL 3x3', 'BMX FREESTYLE', 'BMX RACING', 'Breaking']

df['parsing_epreuve'] = df.apply(parsing_epreuve, axis=1)
#df.to_csv('../data/session_planning_olympics_v10.csv')
# df

# Matching localisations

In [145]:
pd.read_csv("../data/matching_infras4.csv")

,Planning,Infra
0,Trocadéro,NaN
1,Stade de France,Stade de France
2,Hôtel de Ville (site de départ) - Invalides (s...,Hôtel de ville de Paris
3,Stade nautique de Vaires-sur-Marne - Bassin ea...,Stade nautique
4,Arena Porte de La Chapelle,Arena La Chapelle
5,Stade Pierre Mauroy*,NaN
6,Arena Bercy,Arena Bercy
7,La Concorde 1,Stade de la Concorde
8,La Concorde,Stade de la Concorde
9,Stade BMX de Saint-Quentin-en-Yvelines,Stade BMX de Saint-Quentin-en-Yvelines


In [6]:
import pandas as pd

# import data
df1 = pd.read_csv("paris-2024-sites-olympiques-et-paralympiques-franciliens.csv", delimiter=";")
# df = pd.read_csv("../data/session_planning_olympics_v9.csv")
matching_infras = pd.read_csv("matching_infras4.csv")
matching_infras2 = matching_infras.rename(columns={"Planning" : "lieu_epreuve", "Infra" : "NOM"})

# merge
df3 = pd.merge(df, matching_infras2, how='left', on="lieu_epreuve")
df4 = pd.merge(df3, df1, how='left', on="NOM")

# processing localisation
df4.loc[df4['Geo Point'].notna(), 'latitude'] = df4.loc[df4['Geo Point'].notna(), 'Geo Point'].apply(lambda x: float(x.split(', ')[0]))
df4.loc[df4['Geo Point'].notna(), 'longitude'] = df4.loc[df4['Geo Point'].notna(), 'Geo Point'].apply(lambda x: float(x.split(', ')[1]))

# filtrer les nan
print(len(df4.loc[df4['Geo Point'].notna()])/len(df4)) # --> on garde 80% des infras 

df4 = df4.loc[df4['Geo Point'].notna()]
#df4.to_csv('../data/session_planning_pars_with_loc_v12.csv')

0.8190224570673712


In [127]:
#Reste plus que le matching --> NAAN
matching_infras2

,lieu_epreuve,NOM
0,Arena Champ-de-Mars,Arena Champs de Mars
1,Arena Paris Nord,Arena Paris nord
2,Arena Paris Sud 6,Arena Paris Sud 6 (Porte de Versailles)
3,Arena Porte de La Chapelle,Arena La Chapelle
4,Centre National de Tir de Châteauroux*,NaN
5,Départ Invalides - Arrivée Pont Alexandre III,NaN
6,Paris La Défense Arena,La Défense Arena
7,Porte de La Chapelle Arena,Arena La Chapelle
8,Stade Yves-du-Manoir - Terrain 1,Stade Yves du Manoir
9,Stade Yves-du-Manoir - Terrain 2,Stade Yves du Manoir
